In [33]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

## Create a set of 2000 random latitudes and longitudes.

In [36]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# Output is a zip object in memory
coordinates = zip(lats, lngs)
coordinates

## Get the nearest city using the citipy module.

In [37]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

758

## Perform an API call with the OpenWeatherMap.

In [6]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

## Retrieve the following from the API Call:
- Latitude & Longitude
- Maximum Temperature
- Percent Humidity
- Percent Cloudiness
- Wind Speed
- Weather description

In [40]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | esperance
City not found. Skipping...
Processing Record 2 of Set 1 | salamanca
City not found. Skipping...
Processing Record 3 of Set 1 | bredasdorp
City not found. Skipping...
Processing Record 4 of Set 1 | upernavik
City not found. Skipping...
Processing Record 5 of Set 1 | carnarvon
City not found. Skipping...
Processing Record 6 of Set 1 | punta arenas
City not found. Skipping...
Processing Record 7 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 8 of Set 1 | atbasar
City not found. Skipping...
Processing Record 9 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 10 of Set 1 | pemberton
City not found. Skipping...
Processing Record 11 of Set 1 | arraial do cabo
City not found. Skipping...
Processing Record 12 of Set 1 | thunder bay
City not found. Skipping...
Processing Record 13 of Set 1 | floro
City not found. Skipping...
Processing Record 14

In [8]:
len(city_data)

559

## Add the data to a new DataFrame.

In [9]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,71.19,79,71,17.05,RE,2021-07-09 20:45:27
1,Kisangani,0.5167,25.2000,74.73,76,84,2.66,CD,2021-07-09 20:45:27
2,Lazurne,46.0840,32.5295,71.02,60,0,9.06,UA,2021-07-09 20:45:28
3,Georgetown,5.4112,100.3354,76.95,94,20,3.00,MY,2021-07-09 20:45:28
4,Coquimbo,-29.9533,-71.3436,58.73,72,0,10.36,CL,2021-07-09 20:45:05
5,Mataura,-46.1927,168.8643,37.49,97,100,8.16,NZ,2021-07-09 20:45:28
6,Manbij,36.5281,37.9549,86.77,48,0,13.51,SY,2021-07-09 20:45:29
7,Puerto Ayacucho,5.6639,-67.6236,79.77,78,99,2.44,VE,2021-07-09 20:45:29
8,Morant Bay,17.8815,-76.4093,88.63,75,12,13.04,JM,2021-07-09 20:45:29
9,Wolfsburg,52.4333,10.8000,62.31,98,100,8.28,DE,2021-07-09 20:45:30


## Export the DataFrame as a csv file and save it as WeatherPy_Database.csv

In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")